# Background

Data gathered from the [NYT github page](https://github.com/nytimes/covid-19-data), [USDA](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697), and the US Census [population](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/) and [land area](https://www.census.gov/geographies/reference-files/2010/geo/state-area.html) pages.

## NYT Methodology

> To transform raw survey responses into county-level estimates, the survey data was weighted by age and gender, and survey respondents’ locations were approximated from their ZIP codes. Then estimates of mask-wearing were made for each census tract by taking a weighted average of the 200 nearest responses, with closer responses getting more weight in the average. These tract-level estimates were then rolled up to the county level according to each tract’s total population.
> 
> By rolling the estimates up to counties, it reduces a lot of the random noise that is seen at the tract level. In addition, the shapes in the map are constructed from census tracts that have been merged together — this helps in displaying a detailed map, but is less useful than county-level in analyzing the data.

# Setup

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import requests

## Helper Functions

In [2]:
def list_diff(list1,list2):
    '''
    Finds what is missing from, or what is different between, the two lists.
    
    return
    ------
    list_difference: list
    '''
    list_difference = {}
    
    if len(list1) > len(list2):
        bigger = list1
        smaller = list2
        small_list = 'list2'
    else:
        bigger = list2
        smaller = list1
        small_list = 'list1'
        
    for item in bigger:
        if item not in smaller:
            list_difference[item] = f'missing from {small_list}'

    return list_difference

# County Data

## Mask Data

In [3]:
df_mask = pd.read_csv("data/nyt_data/mask_use_data/mask-use-by-county.csv")
df_mask.columns = df_mask.columns.str.lower()
df_mask.head()

,countyfp,never,rarely,sometimes,frequently,always
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


## Covid Data

In [4]:
county_covid = pd.read_csv("data/nyt_data/us-counties.csv",parse_dates=['date'])

In [5]:
county_covid = county_covid.astype({
    'county':str,
    'fips':float,
    'cases':int,
    'deaths':int
})

In [6]:
# Rename columns
county_covid.columns = ['date','county','state','fips','covid_cases','covid_deaths']
# Rearrange columns
county_covid = county_covid[['date','state','county','fips','covid_cases','covid_deaths']]
county_covid.head()

,date,state,county,fips,covid_cases,covid_deaths
0,2020-01-21,Washington,Snohomish,53061.0,1,0
1,2020-01-22,Washington,Snohomish,53061.0,1,0
2,2020-01-23,Washington,Snohomish,53061.0,1,0
3,2020-01-24,Illinois,Cook,17031.0,1,0
4,2020-01-24,Washington,Snohomish,53061.0,1,0


On a previous (now removed) merging of the NYT `df_mask` and NYT `county_covid` datasets we found that the `df_mask` dataset was missing some counties. To double check what these counties are, the [USDA link](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697) was scraped for all fips, name, and state data.

## What was Missing?

In [7]:
response = requests.get("https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697")
soup = BeautifulSoup(response.content)

In [8]:
table_tag = soup.find(class_='data')

county_scrape = pd.DataFrame(columns=['fips','county','state'])

for tr in table_tag.find_all('tr')[1:]:
    tds = tr.find_all('td')
    d = pd.DataFrame(data = {'fips':[tds[0].text], 'county':[tds[1].text], 'state':[tds[2].text]})
    county_scrape = county_scrape.append(d)

In [9]:
county_scrape = county_scrape.reset_index(drop=True)
county_scrape = county_scrape.astype({
    'fips':int,
    'county':str,
    'state':'category'
})

In [10]:
county_scrape.head()

,fips,county,state
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [11]:
missing = list_diff(county_scrape['county'].unique(),county_covid['county'].unique())
missing_df = pd.DataFrame.from_dict(missing,orient='index')
scraped = county_scrape.set_index('county')
missing_df = missing_df.sort_index().reset_index()
missing_from_scrape = scraped.merge(missing_df, left_on='county',right_on='index')

In [12]:
missing_from_scrape.groupby('state').count().sort_values('fips',ascending=False)

,fips,index,0
state,,,
VA,41,41,41
AK,25,25,25
LA,9,9,9
MO,6,6,6
NY,4,4,4
AS,4,4,4
IN,4,4,4
MD,4,4,4
MP,3,3,3


The vast majority of missing counties are from Virginia and Alaska.

In [13]:
missing_from_scrape[(missing_from_scrape['state']=='AS') ]

,fips,state,index,0
122,60020,AS,Manua,missing from list2
123,60020,AS,Ofu,missing from list2
124,60020,AS,Olosega,missing from list2
125,60050,AS,Tutuila,missing from list2


The reasons for missing counties include:

* Virginia's missing counties are all cities
* Alaska: unclear why
* Louisiana: unincorporated communities or parishes
* MO: De Kalb has a population of 220, is part of St. Joseph statistical area.
* MO: St. Francois has a population of 65k, but the county seat is in Farmington. The rest are similar, with the county seat being in another county.
* NY: boroughs of NYC, St. Lawrence has county seat in Canton
* AS: Indian reservation

The rest of the list have 4 or less missing counties, reasons are assumed to be similar as above.

## Merging Mask and County Covid Data

The NYT mask data is merged with the NYT-included `county_covid` dataset. This allows us to see  the number of covid cases per county, along with the reported mask use.

Because `county_covid` is a timeseries, we first filter to include only the latest total cases (from August 1, 2020 onwards) and then take the mean per county. In this way we should have one row per county.

In [14]:
df_mask.head()

,countyfp,never,rarely,sometimes,frequently,always
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [15]:
county_cases_aug_on = county_covid[county_covid['date'] > '20200801'].groupby('fips').mean().reset_index()

In [16]:
county_cases_aug_on

,fips,covid_cases,covid_deaths
0,1001.0,1161.857143,22.142857
1,1003.0,3579.142857,27.571429
2,1005.0,631.571429,5.357143
3,1007.0,432.714286,4.714286
4,1009.0,913.714286,3.928571
...,...,...,...
3193,72151.0,121.285714,0.000000
3194,72153.0,241.714286,0.000000
3195,78010.0,296.571429,4.785714
3196,78020.0,19.071429,0.000000


In [17]:
county_mask = county_cases_aug_on.merge(df_mask,left_on='fips',right_on='countyfp')

In [18]:
county_mask = county_mask[['fips', 'covid_cases', 'covid_deaths', 'never', 'rarely', 'sometimes','frequently', 'always']]
county_mask.columns = ['fips', 'covid_cases', 'covid_deaths', 'mask_never', 'mask_rarely', 'mask_sometimes','mask_frequently', 'mask_always']

In [19]:
county_mask.head()

,fips,covid_cases,covid_deaths,mask_never,mask_rarely,mask_sometimes,mask_frequently,mask_always
0,1001.0,1161.857143,22.142857,0.053,0.074,0.134,0.295,0.444
1,1003.0,3579.142857,27.571429,0.083,0.059,0.098,0.323,0.436
2,1005.0,631.571429,5.357143,0.067,0.121,0.120,0.201,0.491
3,1007.0,432.714286,4.714286,0.020,0.034,0.096,0.278,0.572
4,1009.0,913.714286,3.928571,0.053,0.114,0.180,0.194,0.459


So far we:

* Imported and cleaned `df_mask` dataframe. It includes survey results of how often people wear masks in each county.
* Import and cleaned the `county_covid` dataframe, then found the average for data past Aug 1, 2020. This includes total (not new) covid19 cases and deaths.
* The two dataframes were merged into one `county_mask` dataframe. This includes all NYT data to date (Aug 16, 2020).

__Problem:__
After using the `groupby()` function all `str` type columns were removed (as you cannot take the mean of strings). We lost the county and state names, but the `fips` values remained.

__Solution:__ We can merge our new `county_mask` dataset with another dataset to get these names back. Since we want to find the population per county anyways, we will use the [US Census 2019 population estimate](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html) dataset to get county name, state name, county population, and some other variables.

## County Population

__NOTE:__ last 6 rows of the raw excel file contain disclaimers and citing instructions

__NOTE:__ When shown without a date variable, county covid data from here on will reflect the mean of only "recent" (August 1, 2020 onwards) figures.

In [20]:
countypop = pd.read_excel('data/added_data/countypop.xlsx')
countypop.columns = countypop.columns.str.lower()

The following keys were provided in a separate pdf file by the US census, letting us map them for interpretation.

In [21]:
region_key = {
    1:'northeast',
    2:'midwest',
    3:'south',
    4:'west'
}
division_key = {
    1:'new_england',
    2:'middle_atlantic',
    3:'east_north_central',
    4:'west_north_central',
    5:'south_atlantic',
    6:'east_south_central',
    7:'west_south_central',
    8:'mountain',
    9:'pacific'
}
sumlev_key = {
    40:'state_or_equiv',
    50:'county_or_equiv'
}

In [22]:
# select only the columns that are relevant, which is the latest (2019) estimates
countypop = countypop[['sumlev','region','division','state','county','stname','ctyname','popestimate2019',
                       'births2019','internationalmig2019','domesticmig2019','rbirth2019','rdeath2019']]
countypop.columns = ['sumlev','region_fips','division_fips','state_fips','county_fips','state','county',
                     'population','births','intnl_migration','domestic_migration','birth_rate','death_rate']
countypop.head()

,sumlev,region_fips,division_fips,state_fips,county_fips,state,county,population,births,intnl_migration,domestic_migration,birth_rate,death_rate
0,40,3,6,1,0,Alabama,Alabama,4903185,57313,2772,9387,11.707442,11.005972
1,50,3,6,1,1,Alabama,Autauga County,55869,624,-16,270,11.202671,9.712572
2,50,3,6,1,3,Alabama,Baldwin County,223234,2304,80,5297,10.446871,10.546624
3,50,3,6,1,5,Alabama,Barbour County,24686,256,13,-141,10.331329,12.591307
4,50,3,6,1,7,Alabama,Bibb County,22394,240,10,31,10.723621,11.259802


In [23]:
# find the counties that are present in our dataset
cty_fip = county_covid[['state','county','fips']].groupby(['state','county']).mean().reset_index()
cty_fip.head()

,state,county,fips
0,Alabama,Autauga,1001.0
1,Alabama,Baldwin,1003.0
2,Alabama,Barbour,1005.0
3,Alabama,Bibb,1007.0
4,Alabama,Blount,1009.0


It looks like the two datasets have very similar naming styles, with the exception that the NYT `county_covid` dataset does not include the words `County` or `Parish` after each territory. These are removed from the Census dataset, along with any spaces, and then merged on relevant county and state names.

In [24]:
# format for merging
countypop['county'] = countypop['county'].str.replace('County','')
countypop['county'] = countypop['county'].str.replace('Parish','')
countypop['county'] = countypop['county'].str.replace(' ','')

In [25]:
# merge fips from covid dataset with 2019pop
cty_pop = cty_fip.merge(countypop,left_on=['state','county'],right_on=['state','county'])

In [26]:
cty_pop = cty_pop[['state', 'county', 'fips', 'sumlev', 'region_fips', 'division_fips',
       'population','births', 'intnl_migration', 'domestic_migration', 'birth_rate',
       'death_rate']]

In [27]:
cty_pop.head()

,state,county,fips,sumlev,region_fips,division_fips,population,births,intnl_migration,domestic_migration,birth_rate,death_rate
0,Alabama,Autauga,1001.0,50,3,6,55869,624,-16,270,11.202671,9.712572
1,Alabama,Baldwin,1003.0,50,3,6,223234,2304,80,5297,10.446871,10.546624
2,Alabama,Barbour,1005.0,50,3,6,24686,256,13,-141,10.331329,12.591307
3,Alabama,Bibb,1007.0,50,3,6,22394,240,10,31,10.723621,11.259802
4,Alabama,Blount,1009.0,50,3,6,57826,651,6,59,11.263268,11.367077


## Merging Mask with County Pop

We now have two datasets:

* `county_mask` that contains all NYT data regarding covid19 cases and mask use
* `cty_pop` that contains US Census data about the population

If we can merge these dataframes, we can find out the number of cases per population and a bunch of other interesting statistics. So that's what we will do next.

In [28]:
list_diff(county_covid['county'].unique(),cty_pop['county'].unique())

{'Los Angeles': 'missing from list2',
 'Santa Clara': 'missing from list2',
 'San Francisco': 'missing from list2',
 'San Diego': 'missing from list2',
 'Salt Lake': 'missing from list2',
 'New York City': 'missing from list2',
 'Unknown': 'missing from list2',
 'San Mateo': 'missing from list2',
 'Walla Walla': 'missing from list2',
 'Contra Costa': 'missing from list2',
 'Fort Bend': 'missing from list2',
 'Santa Rosa': 'missing from list2',
 'El Paso': 'missing from list2',
 'Santa Cruz': 'missing from list2',
 'District of Columbia': 'missing from list2',
 'St. Louis': 'missing from list2',
 "Prince George's": 'missing from list2',
 'Virginia Beach city': 'missing from list2',
 'San Joaquin': 'missing from list2',
 'Charles Mix': 'missing from list2',
 'New Castle': 'missing from list2',
 'St. Johns': 'missing from list2',
 'St. Joseph': 'missing from list2',
 'Santa Fe': 'missing from list2',
 'Bon Homme': 'missing from list2',
 'Fond du Lac': 'missing from list2',
 'Anchorage': '

From our list it looks like the following are not included in our `cty_pop` Census dataset, but are in the NYT `county_mask` dataset:

* Cities (ex: Los Angeles, Walla Walla)
* Commonwealth areas (ex: Saipan) are not included
* Some counties (ex: Roger Mills County in Oklahoma)

Some other areas (ex: Roger Mills County) are also not included.

We will ignore these for now, but still keep them in our dataframe by doing a left merge. This will keep all rows in the `county_mask` dataframe even though they have no corresponding data in the `cty_pop` dataframe.

In [29]:
df_county = county_mask.merge(cty_pop, on=['fips'], how='left')

In [30]:
100 * (sum(df_county['population'].isna()) / df_county.shape[0])

7.818007049022749

Approximately 8% of the NYT dataframe has no corresponding data in the Census dataframe. This is acceptable to us for now, so we will go ahead with the analysis.

In [31]:
df_county.head()

,fips,covid_cases,covid_deaths,mask_never,mask_rarely,mask_sometimes,mask_frequently,mask_always,state,county,sumlev,region_fips,division_fips,population,births,intnl_migration,domestic_migration,birth_rate,death_rate
0,1001.0,1161.857143,22.142857,0.053,0.074,0.134,0.295,0.444,Alabama,Autauga,50.0,3.0,6.0,55869.0,624.0,-16.0,270.0,11.202671,9.712572
1,1003.0,3579.142857,27.571429,0.083,0.059,0.098,0.323,0.436,Alabama,Baldwin,50.0,3.0,6.0,223234.0,2304.0,80.0,5297.0,10.446871,10.546624
2,1005.0,631.571429,5.357143,0.067,0.121,0.120,0.201,0.491,Alabama,Barbour,50.0,3.0,6.0,24686.0,256.0,13.0,-141.0,10.331329,12.591307
3,1007.0,432.714286,4.714286,0.020,0.034,0.096,0.278,0.572,Alabama,Bibb,50.0,3.0,6.0,22394.0,240.0,10.0,31.0,10.723621,11.259802
4,1009.0,913.714286,3.928571,0.053,0.114,0.180,0.194,0.459,Alabama,Blount,50.0,3.0,6.0,57826.0,651.0,6.0,59.0,11.263268,11.367077


We map the given keys to their appropriate values for ease of categorization in the future

In [32]:
df_county['region'] = df_county['region_fips'].map(region_key)
df_county['division'] = df_county['division_fips'].map(division_key)
df_county['area_type'] = df_county['sumlev'].map(sumlev_key)

And add some preliminary per capita calculations

In [33]:
df_county['cases_per_million'] = (df_county['covid_cases']/df_county['population']) * 1000000
df_county['cases_per_hthousand'] = (df_county['covid_cases']/df_county['population']) * 100000
df_county['cases_per_thousand'] = (df_county['covid_cases']/df_county['population']) * 1000
df_county['cases_per_hundred'] = (df_county['covid_cases']/df_county['population']) * 100

In [34]:
# rearrange the columns
df_county = df_county[['state','region', 'county', 'division', 'area_type',
                       'population', 'covid_cases', 'covid_deaths', 'cases_per_million', 'cases_per_hthousand', 
                       'cases_per_thousand', 'cases_per_hundred',
                       'mask_never', 'mask_rarely','mask_sometimes', 'mask_frequently', 'mask_always', 
                       'births','intnl_migration', 'domestic_migration', 
                       'birth_rate', 'death_rate',
                       'fips', 'sumlev', 'region_fips', 'division_fips'
                    ]]

In [35]:
df_county.head()

,state,region,county,division,area_type,population,covid_cases,covid_deaths,cases_per_million,cases_per_hthousand,...,mask_always,births,intnl_migration,domestic_migration,birth_rate,death_rate,fips,sumlev,region_fips,division_fips
0,Alabama,south,Autauga,east_south_central,county_or_equiv,55869.0,1161.857143,22.142857,20796.096992,2079.609699,...,0.444,624.0,-16.0,270.0,11.202671,9.712572,1001.0,50.0,3.0,6.0
1,Alabama,south,Baldwin,east_south_central,county_or_equiv,223234.0,3579.142857,27.571429,16033.143953,1603.314395,...,0.436,2304.0,80.0,5297.0,10.446871,10.546624,1003.0,50.0,3.0,6.0
2,Alabama,south,Barbour,east_south_central,county_or_equiv,24686.0,631.571429,5.357143,25584.194627,2558.419463,...,0.491,256.0,13.0,-141.0,10.331329,12.591307,1005.0,50.0,3.0,6.0
3,Alabama,south,Bibb,east_south_central,county_or_equiv,22394.0,432.714286,4.714286,19322.777785,1932.277778,...,0.572,240.0,10.0,31.0,10.723621,11.259802,1007.0,50.0,3.0,6.0
4,Alabama,south,Blount,east_south_central,county_or_equiv,57826.0,913.714286,3.928571,15801.097875,1580.109787,...,0.459,651.0,6.0,59.0,11.263268,11.367077,1009.0,50.0,3.0,6.0


# State Data

## Covid Data

In [36]:
state_covid = pd.read_csv("data/nyt_data/us-states.csv",parse_dates=['date'])

In [37]:
state_covid = state_covid.astype({
    'state':str,
    'fips':float,
    'cases':int,
    'deaths':int
})

In [38]:
# Rename columns
state_covid.columns = ['date','state','state_fips','covid_cases','covid_deaths']
state_covid.head()

,date,state,state_fips,covid_cases,covid_deaths
0,2020-01-21,Washington,53.0,1,0
1,2020-01-22,Washington,53.0,1,0
2,2020-01-23,Washington,53.0,1,0
3,2020-01-24,Illinois,17.0,1,0
4,2020-01-24,Washington,53.0,1,0


We will do the same sort of filtering as for the county data by only including total covid cases since August 1, 2020

In [39]:
state_covid = state_covid[state_covid['date'] >= '20200801'].reset_index(drop=True)
state_covid = state_covid.groupby('state_fips').mean().reset_index()

## Land Area

It would be nice to know the county and state land areas, so we can get some sort of estimate for the population density. I was unable to find information for every county, however information about state land area was found at this [US Census source](https://www.census.gov/geographies/reference-files/2010/geo/state-area.html).

We will scrape and format this data to get it ready for a future merge.

In [40]:
response = requests.get("https://www.census.gov/geographies/reference-files/2010/geo/state-area.html")

In [41]:
soup = BeautifulSoup(response.content)

In [42]:
table_tag = soup.find('tbody')

state_land_scrape = pd.DataFrame(columns=range(1,17))

for tr in table_tag.find_all('tr')[3:]:
    tds = tr.find_all('td')
    d = {}
    for i in range(0,17):
        d[i] = [tds[i].text]
    data = pd.DataFrame.from_dict(data=d)
    state_land_scrape = state_land_scrape.append(data)

In [43]:
cols = ['state']
areas = ['total_area_','land_area_','total_water_area_','inland_water_area_','coastal_water_area_',
         'great_lakes_water_area_','territorial_water_area_','latitude','longitude']
for i in range(1,17):
    if (i in range(1,16)) & (i % 2 == 0):
        unit = 'sqkm'
    elif (i in range(1,16)) & (i % 2 != 0):
        unit = 'sqmi'
    if (i == 1) | (i == 2):
        cols.append(f'total_area_{unit}')
    elif (i == 3) | (i == 4):
        cols.append(f'land_area_{unit}')
    elif (i == 5) | (i == 6):
        cols.append(f'total_water_area_{unit}')
    elif (i == 7) | (i == 8):
        cols.append(f'inland_water_area_{unit}')
    elif (i == 9) | (i == 10):
        cols.append(f'coastal_water_area_{unit}')
    elif (i == 11) | (i == 12):
        cols.append(f'great_lakes_water_area_{unit}')
    elif (i == 13) | (i == 14):
        cols.append(f'territorial_water_area_{unit}')
    elif (i == 15):
        cols.append('latitude')
    elif (i == 16):
        cols.append('longitude')

In [44]:
state_land_scrape.columns=cols
state_land_scrape = state_land_scrape.reset_index(drop=True)
state_land_scrape = state_land_scrape.iloc[3:,:].reset_index(drop=True)

In [45]:
state_land_scrape.head()

,state,total_area_sqmi,total_area_sqkm,land_area_sqmi,land_area_sqkm,total_water_area_sqmi,total_water_area_sqkm,inland_water_area_sqmi,inland_water_area_sqkm,coastal_water_area_sqmi,coastal_water_area_sqkm,great_lakes_water_area_sqmi,great_lakes_water_area_sqkm,territorial_water_area_sqmi,territorial_water_area_sqkm,latitude,longitude
0,Alabama,"52,420","135,767","50,645","131,171","1,775","4,597","1,058","2,740",517,"1,340",—,—,199,516,+32.7396323,-086.8434593
1,Alaska,"665,384","1,723,337","570,641","1,477,953","94,743","245,383","19,304","49,997","26,119","67,647",—,—,"49,320","127,739",+63.3461910,-152.8370679
2,Arizona,"113,990","295,234","113,594","294,207",396,"1,026",396,"1,026",—,—,—,—,—,—,+34.2099643,-111.6024010
3,Arkansas,"53,179","137,732","52,035","134,771","1,143","2,961","1,143","2,961",—,—,—,—,—,—,+34.8955256,-092.4446262
4,California,"163,695","423,967","155,779","403,466","7,916","20,501","2,833","7,339",245,634,—,—,"4,837","12,528",+37.1485730,-119.5406515


## State Pop

We could just figure out the state population from the county data, but the [US census](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/) has this precompiled for us.

In [46]:
statepop_raw = pd.read_csv('data/added_data/statepop.csv')
statepop_raw.columns = statepop_raw.columns.str.lower()

In [47]:
statepop_raw.head()

,sumlev,region,division,state,name,census2010pop,estimatesbase2010,popestimate2010,popestimate2011,popestimate2012,...,rdomesticmig2019,rnetmig2011,rnetmig2012,rnetmig2013,rnetmig2014,rnetmig2015,rnetmig2016,rnetmig2017,rnetmig2018,rnetmig2019
0,10,0,0,0,United States,308745538,308758105,309321666,311556874,313830990,...,0.000000,2.493773,2.682083,2.636187,2.921500,3.260435,3.252788,2.871957,2.153911,1.818059
1,20,1,0,0,Northeast Region,55317240,55318443,55380134,55604223,55775216,...,-5.254530,0.887909,-0.038355,-0.469783,-0.986097,-2.061965,-2.490484,-1.837048,-2.134447,-2.859713
2,20,2,0,0,Midwest Region,66927001,66929725,66974416,67157800,67336743,...,-2.365881,-0.963930,-0.973943,-0.006924,-0.762969,-1.388437,-1.241784,-0.557370,-0.922755,-1.111173
3,20,3,0,0,South Region,114555744,114563030,114866680,116006522,117241208,...,3.261349,5.130513,5.850458,5.292073,6.161501,7.277358,7.150074,6.198168,5.225519,5.203720
4,20,4,0,0,West Region,71945553,71946907,72100436,72788329,73477823,...,0.614245,2.723344,3.062896,3.162262,4.026429,4.987285,5.261078,4.021194,3.044951,2.312083


In [48]:
statepop = statepop_raw[['name','popestimate2019','sumlev','region','division','state']].reset_index(drop=True)
statepop.columns = ['state', 'population','sumlev','region_fips','division_fips','state_fips']

In [49]:
statepop.head()

,state,population,sumlev,region_fips,division_fips,state_fips
0,United States,328239523,10,0,0,0
1,Northeast Region,55982803,20,1,0,0
2,Midwest Region,68329004,20,2,0,0
3,South Region,125580448,20,3,0,0
4,West Region,78347268,20,4,0,0


In [50]:
# 0 represents territories and regions, which we are not interested in. 
# This data is also included in our county datasets.
statepop = statepop[statepop['state_fips']!=0].reset_index(drop=True)

In [51]:
statepop.head()

,state,population,sumlev,region_fips,division_fips,state_fips
0,Alabama,4903185,40,3,6,1
1,Alaska,731545,40,4,9,2
2,Arizona,7278717,40,4,8,4
3,Arkansas,3017804,40,3,7,5
4,California,39512223,40,4,9,6


## Merge Land Area with State Population

By merging the two we can get population density data.

In [52]:
# Merge population with land area
statepop = statepop.merge(state_land_scrape, on='state')

In [53]:
statepop = statepop.replace(to_replace = '—', value = np.nan)
statepop = statepop.replace(to_replace = ',', value = '')

In [54]:
statepop.head()

,state,population,sumlev,region_fips,division_fips,state_fips,total_area_sqmi,total_area_sqkm,land_area_sqmi,land_area_sqkm,...,inland_water_area_sqmi,inland_water_area_sqkm,coastal_water_area_sqmi,coastal_water_area_sqkm,great_lakes_water_area_sqmi,great_lakes_water_area_sqkm,territorial_water_area_sqmi,territorial_water_area_sqkm,latitude,longitude
0,Alabama,4903185,40,3,6,1,"52,420","135,767","50,645","131,171",...,"1,058","2,740",517,"1,340",NaN,NaN,199,516,+32.7396323,-086.8434593
1,Alaska,731545,40,4,9,2,"665,384","1,723,337","570,641","1,477,953",...,"19,304","49,997","26,119","67,647",NaN,NaN,"49,320","127,739",+63.3461910,-152.8370679
2,Arizona,7278717,40,4,8,4,"113,990","295,234","113,594","294,207",...,396,"1,026",NaN,NaN,NaN,NaN,NaN,NaN,+34.2099643,-111.6024010
3,Arkansas,3017804,40,3,7,5,"53,179","137,732","52,035","134,771",...,"1,143","2,961",NaN,NaN,NaN,NaN,NaN,NaN,+34.8955256,-092.4446262
4,California,39512223,40,4,9,6,"163,695","423,967","155,779","403,466",...,"2,833","7,339",245,634,NaN,NaN,"4,837","12,528",+37.1485730,-119.5406515


## Merge with State Covid Data

In [55]:
df_state = state_covid.merge(statepop, on='state_fips')

We replicate what we did to `df_county` with `df_state`

In [56]:
cols = ['region_fips', 'division_fips',
       'total_area_sqmi', 'total_area_sqkm', 'land_area_sqmi',
       'land_area_sqkm', 'total_water_area_sqmi', 'total_water_area_sqkm',
       'inland_water_area_sqmi', 'inland_water_area_sqkm',
       'coastal_water_area_sqmi', 'coastal_water_area_sqkm',
       'great_lakes_water_area_sqmi', 'great_lakes_water_area_sqkm',
       'territorial_water_area_sqmi', 'territorial_water_area_sqkm',
       'latitude', 'longitude']

In [57]:
for col in cols:
    df_state[col] = df_state[col].str.replace('X','NaN')
    df_state[col] = df_state[col].str.replace(',','')
    df_state[col] = df_state[col].astype(float)

In [58]:
df_state['region'] = df_state['region_fips'].map(region_key)
df_state['division'] = df_state['division_fips'].map(division_key)
df_state['area_type'] = df_state['sumlev'].map(sumlev_key)

And add some preliminary per capita calculations

In [59]:
df_state['cases_per_million'] = (df_state['covid_cases']/df_state['population']) * 1000000
df_state['cases_per_hthousand'] = (df_state['covid_cases']/df_state['population']) * 100000
df_state['cases_per_thousand'] = (df_state['covid_cases']/df_state['population']) * 1000
df_state['cases_per_hundred'] = (df_state['covid_cases']/df_state['population']) * 100

And rearrange the columns

In [60]:
df_state = df_state[['state', 'region', 'division', 'area_type','covid_cases', 'covid_deaths', 'population',
                    'cases_per_million', 'cases_per_hthousand', 'cases_per_thousand',
                    'cases_per_hundred', 'state_fips','sumlev', 'region_fips', 'division_fips', 
                    'total_area_sqmi','total_area_sqkm', 'land_area_sqmi', 'land_area_sqkm',
                    'total_water_area_sqmi', 'total_water_area_sqkm',
                    'inland_water_area_sqmi', 'inland_water_area_sqkm',
                    'coastal_water_area_sqmi', 'coastal_water_area_sqkm',
                    'great_lakes_water_area_sqmi', 'great_lakes_water_area_sqkm',
                    'territorial_water_area_sqmi', 'territorial_water_area_sqkm',
                    'latitude', 'longitude']]

In [61]:
df_state.head()

,state,region,division,area_type,covid_cases,covid_deaths,population,cases_per_million,cases_per_hthousand,cases_per_thousand,...,inland_water_area_sqmi,inland_water_area_sqkm,coastal_water_area_sqmi,coastal_water_area_sqkm,great_lakes_water_area_sqmi,great_lakes_water_area_sqkm,territorial_water_area_sqmi,territorial_water_area_sqkm,latitude,longitude
0,Alabama,south,east_south_central,state_or_equiv,99287.533333,1760.066667,4903185,20249.599665,2024.959966,20.249600,...,1058.0,2740.0,517.0,1340.0,NaN,NaN,199.0,516.0,32.739632,-86.843459
1,Alaska,west,pacific,state_or_equiv,4391.066667,23.733333,731545,6002.455989,600.245599,6.002456,...,19304.0,49997.0,26119.0,67647.0,NaN,NaN,49320.0,127739.0,63.346191,-152.837068
2,Arizona,west,mountain,state_or_equiv,185412.733333,4099.333333,7278717,25473.271365,2547.327137,25.473271,...,396.0,1026.0,NaN,NaN,NaN,NaN,NaN,NaN,34.209964,-111.602401
3,Arkansas,south,west_south_central,state_or_equiv,48281.200000,531.533333,3017804,15998.785872,1599.878587,15.998786,...,1143.0,2961.0,NaN,NaN,NaN,NaN,NaN,NaN,34.895526,-92.444626
4,California,west,pacific,state_or_equiv,560676.533333,10266.200000,39512223,14189.951634,1418.995163,14.189952,...,2833.0,7339.0,245.0,634.0,NaN,NaN,4837.0,12528.0,37.148573,-119.540651


# Save to DB

Finally, both `county_covid`,`df_county`, `df_state` are saved into an sqlite database for ease of access:

```python
import sqlalchemy as sq
location = 'sqlite:///data/nyt_covid.db'
cnx = sq.create_engine(location)

# save county time series
county_covid.to_sql('county_covid_dates', con=cnx, if_exists='fail', index=False)
# save county
df_county.to_sql('county_dataset', con=cnx, if_exists='fail', index=False)
# save state
df_state.to_sql('state_dataset', con=cnx, if_exists='fail', index=False)
```

To access in the future we just need to run:

```python
import pandas as pd
import sqlalchemy as sq

location = 'sqlite:///data/nyt_covid.db'
cnx = sq.create_engine(location)

county_covid = pd.read_sql_table('county_covid_dates',cnx)
df_county = pd.read_sql_table('county_dataset',cnx)
df_state = pd.read_sql_table('state_dataset',cnx)
```